# Introduction 
L'objectif du notebook, comparé à la semaine dernière est de rajouter des indicateurs pertinents en modifiant la liste précédemment établie :  

- **marketCap** - Capitalisation boursière
- **marginProfit** - Marge nette
- **roe** - Rentabilité des fonds propres (%)
- **roic** - Rentabilité des capitaux investis (%)
- **revenuePerShare** - Chiffre d'affaires par action
- **netIncomePerShare** - Bénéfice net par action
- **debtToEquity** - D/E Ratio
- **peRatio** - P/E Ratio
- **shareValue** - Valeur de l'action  

Et d'ajouter des indicateurs de croissance annuelle dans les cas où ils sont pertients :   
- **marginProfitYoYGrowth** - Croissance annuelle de la marge nette (%)
- **roeYoYGrowth** - Croissance annuelle du ROE (%)
- **roicYoYGrowth** - Croissance annuelle du ROIC (%)
- **revenuePerShareYoYGrowth** - Croissance annuelle du chiffre d'affaires par action (%)
- **netIncomePerShareYoYGrowth** - Croissance annuelle des bénéfices nets (%)
- **debtToEquityYoYGrowth** - Croissance annuelle du D/E Ratio (%)

Ces indicateurs seront évalués sous la forme de time series sur 5 ans de 2020 à 2024.

In [1]:
# Import des librairies et données
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings("ignore")

price = pd.read_csv('../csv/historical_ohlc_nasdaq100.csv')
fundamental = pd.read_csv('../csv/nasdaq100_fundamentals_fpm.csv')

display(price.head())
display(fundamental.head())

,date,open,high,low,close,adjClose,volume,unadjustedVolume,change,changePercent,vwap,label,changeOverTime,symbol
0,2020-01-02,330.00,334.48,329.170,334.43,334.429993,1990496,1990100,4.43,1.34000,332.94,"January 02, 20",0.013400,ADBE
1,2020-01-03,329.17,332.98,328.690,331.81,331.809998,1579371,1577600,2.64,0.80202,331.84,"January 03, 20",0.008020,ADBE
2,2020-01-06,328.29,333.91,328.200,333.71,333.709991,1875122,1874700,5.42,1.65000,332.42,"January 06, 20",0.016500,ADBE
3,2020-01-07,334.15,334.79,332.305,333.39,333.390000,2507261,2500800,-0.76,-0.22744,333.80,"January 07, 20",-0.002274,ADBE
4,2020-01-08,333.81,339.23,333.410,337.87,337.869995,2248531,2248500,4.06,1.22000,337.20,"January 08, 20",0.012200,ADBE


,symbol,date,calendarYear,period,revenuePerShare,netIncomePerShare,operatingCashFlowPerShare,freeCashFlowPerShare,cashPerShare,bookValuePerShare,...,averagePayables,averageInventory,daysSalesOutstanding,daysPayablesOutstanding,daysOfInventoryOnHand,receivablesTurnover,payablesTurnover,inventoryTurnover,roe,capexPerShare
0,ADBE,2024-11-29,2024,FY,48.109620,12.438479,18.022371,17.503356,17.642058,31.554810,...,337500000.0,0.0,35.167635,55.879983,0.0,10.378861,6.531856,0.0,0.394186,0.519016
1,ADBE,2023-12-01,2023,FY,42.470460,11.877462,15.978118,15.190372,17.159737,36.144420,...,346500000.0,0.0,41.823896,48.687341,0.0,8.727068,7.496815,0.0,0.328611,0.787746
2,ADBE,2022-12-02,2022,FY,37.459574,10.119149,16.676596,15.736170,12.970213,29.895745,...,345500000.0,0.0,42.810690,63.896074,0.0,8.525908,5.712401,0.0,0.338481,0.940426
3,ADBE,2021-12-03,2021,FY,33.092243,10.109015,15.142558,14.450734,12.155136,31.020964,...,309000000.0,0.0,43.425404,61.061662,0.0,8.405218,5.977564,0.0,0.325877,0.691824
4,ADBE,2020-11-27,2020,FY,26.752599,10.935551,11.906445,11.035343,12.457380,27.575884,...,257500000.0,0.0,39.654181,64.860627,0.0,9.204578,5.627451,0.0,0.396562,0.871102


# Merging

In [2]:
# Conversion des colonnes date en type datetime
price.date = pd.to_datetime(price.date)
fundamental.date = pd.to_datetime(fundamental.date)

# Ajout des valeurs 'close' (valeur à la cloture) à fundamental
df = fundamental.merge(price[['symbol', 'date', 'close']], on = ['symbol', 'date'], how='left')
df.close.isna().sum()

# On voit que trop de valeurs (la moitié) sont manquantes

np.int64(254)

In [3]:
# Trie les DF pour merge_asof
fundamental = fundamental.sort_values(by=['date', 'symbol'])
price = price.sort_values(by=['date', 'symbol'])

# Merging à la date la plus proche pour éviter les valeurs manquantes
df = pd.merge_asof(fundamental, 
                   price[['symbol', 'date', 'close']], # On garde uniquement les colonnes qui nous intéressent
                   on='date',
                   by='symbol',
                   direction='backward')

# Cleaning

In [4]:
# Gestion des cas particuliers de valeurs manquantes restantes
# price[price.symbol =='CCEP'] - date la plus proche (3 jours de décalage)
df.loc[0, 'close'] = 50.93

# price[price.symbol =='PDD'] -  date la plus proche (3 jours de décalage)
df.loc[1, 'close'] = 41.23

# price[price.symbol =='APP'] - Pas de valeur proche - entrée en bourse plus tard
df.loc[37, 'close'] = 65.20   # On ajoute, dans ce cas, tout de même la valeur de l'entrée en bourse 4 mois plus tard

# price[price.symbol =='CEG'] -  Pas de valeur proche pour l'année 2020, on ajoute à 2021
df.loc[147, 'close'] = 42.00 # 19 jours d'écart

# price[price.symbol =='GEHC'] # Pas de valeur proche
# price[price.symbol =='ARM']  # Pas de valeur proche
# Pour le reste, on supprimmera les NaNs

# On supprimme Alphabet catégorie C
df = df[df.symbol != 'GOOG']

# On crée marginProfit
df['marginProfit'] = df.netIncomePerShare / df.revenuePerShare

# On renomme close pour refléter la variable dans le contexte du df fundamental
df = df.rename(columns={'close':'shareValue'})
columns_of_interest = ['symbol',
                       'calendarYear', 
                       'marketCap', 
                       'marginProfit', 
                       'roe',
                       'roic',
                       'revenuePerShare',
                       'netIncomePerShare',
                       'debtToEquity',
                       'peRatio',
                       'shareValue']

clean_df = df[columns_of_interest]

# On supprimme les valeurs NaNs (Entreprises entrées en bourses plus tard)
clean_df = clean_df.dropna()

In [5]:
# On remarque des outliers au niveau de min / max par rapport au Q1 et Q3 (IQR)
clean_df.describe()

,calendarYear,marketCap,marginProfit,roe,roic,revenuePerShare,netIncomePerShare,debtToEquity,peRatio,shareValue
count,492.000000,4.920000e+02,492.000000,492.000000,492.000000,492.000000,492.000000,492.000000,492.000000,492.000000
mean,2022.048780,2.239000e+11,0.115875,0.264621,0.122093,44.796351,5.916388,0.535395,-167.145055,229.830935
std,1.444724,6.000375e+11,0.275173,2.929921,0.172418,80.382119,12.748441,10.433174,5460.294237,375.383423
min,2019.000000,1.603054e+09,-2.943927,-32.589389,-1.309052,0.609703,-29.574113,-194.549374,-117858.733542,6.420000
25%,2021.000000,3.645430e+10,0.070630,0.057754,0.043943,12.846238,1.121912,0.182910,16.718417,66.227500
50%,2022.000000,5.496283e+10,0.148548,0.187318,0.109118,23.786916,3.534687,0.494927,27.749101,135.755000
75%,2023.000000,1.388596e+11,0.231964,0.332212,0.197180,43.013659,7.384398,1.216585,46.505323,253.210000
max,2025.000000,6.031718e+12,0.864708,53.038744,1.969265,706.055559,174.944977,43.952473,26277.621897,4968.420000


In [6]:
# On affiche la distribution des variables
indicators = ['marketCap', 
              'marginProfit', 
              'roe',
              'roic',
              'revenuePerShare',
              'netIncomePerShare',
              'debtToEquity',
              'peRatio',
              'shareValue']

fig = make_subplots(rows = 3, cols = 3)
for i, indicator in enumerate(indicators):
    box = px.strip(clean_df[indicator], 
                   hover_name=clean_df['symbol']) 
    
    # Division integer et modulo pour indiquer les rows/cols dans la boucle     
    fig.add_trace(box.data[0], row=i//3+1, col=(i%3)+1) 

fig.update_layout(title='Distribution des variables', 
                  width=1300, 
                  height=900)\
                  .show(renderer='iframe') # à changer en .show('notebook') ou .show() si non fonctionnel

In [7]:
# On retire les outliers car sûrement dûs à des erreurs dans la donnée
symbols_to_filter = ['PDD', 'MELI', 'VRSK', 'MDB', 'ORLY', 'MSTR']
filter_series = ~clean_df['symbol'].isin(symbols_to_filter)
clean_df = clean_df[filter_series]

In [8]:
# On crée 5 lignes, une par année pour l'indice NASDAQ 
# et comparer des entreprises individuelles à la moyenne pondérée de l'indice 

years = [2020, 2021, 2022, 2023, 2024] 

# Fonction de calcul de la moyenne pondérée
def mean_nasdaq(metric, df):
    return (df[metric] * df.marketCap).sum() / df.marketCap.sum()

for year in years:
    nsdq = clean_df[clean_df.calendarYear == year]
    
    n_data = {
    'symbol'            : ['NASDAQ'],
    'calendarYear'      : year, 
    'marketCap'         : [mean_nasdaq('marketCap', nsdq)], 
    'marginProfit'      : [mean_nasdaq('marginProfit', nsdq)], 
    'roe'               : [mean_nasdaq('roe', nsdq)],
    'roic'              : [mean_nasdaq('roic', nsdq)],
    'revenuePerShare'   : [mean_nasdaq('revenuePerShare', nsdq)],
    'netIncomePerShare' : [mean_nasdaq('netIncomePerShare', nsdq)],
    'debtToEquity'      : [mean_nasdaq('debtToEquity', nsdq)],
    'peRatio'           : [mean_nasdaq('peRatio', nsdq)],
    'shareValue'        : [mean_nasdaq('shareValue', nsdq)]}
    
    nasdaq_df = pd.DataFrame(n_data, columns=columns_of_interest)
    clean_df = pd.concat([clean_df, nasdaq_df], axis=0)

In [9]:
# On vérifie le bon ajout des valeurs
display(clean_df.isna().sum())
clean_df

symbol               0
calendarYear         0
marketCap            0
marginProfit         0
roe                  0
roic                 0
revenuePerShare      0
netIncomePerShare    0
debtToEquity         0
peRatio              0
shareValue           0
dtype: int64

,symbol,calendarYear,marketCap,marginProfit,roe,roic,revenuePerShare,netIncomePerShare,debtToEquity,peRatio,shareValue
0,CCEP,2019,2.114228e+10,0.090705,0.176550,0.080541,25.712871,2.332282,1.043047,19.452923,50.930000
2,LULU,2019,3.212492e+10,0.162239,0.330699,0.229307,30.517712,4.951155,0.379036,49.760103,239.390000
3,EA,2020,2.934981e+10,0.548853,0.407318,0.319443,18.897611,10.372014,0.133494,9.657720,100.170000
4,MCHP,2020,1.619742e+10,0.108187,0.102157,0.149314,11.038510,1.194224,1.697646,28.386646,33.900000
5,TTWO,2020,1.341432e+10,0.130937,0.159283,0.126726,27.312814,3.576245,0.069803,33.166073,118.610000
...,...,...,...,...,...,...,...,...,...,...,...
0,NASDAQ,2020,8.965772e+11,0.172628,0.321242,0.148748,30.984720,4.223135,0.975355,94.333044,190.790349
0,NASDAQ,2021,1.124789e+12,0.213482,0.448293,0.197215,35.520955,6.178423,0.868810,46.720408,232.759278
0,NASDAQ,2022,9.487101e+11,0.208751,0.590143,0.215430,41.614554,6.360706,1.246786,14.854333,175.594318
0,NASDAQ,2023,1.213803e+12,0.210058,0.422199,0.203884,45.389697,7.761861,0.671955,39.816742,249.077927


# Ajout des indicateurs de croissance

In [10]:
# Tri du df
df_sorted = clean_df.sort_values(by=['symbol', 'calendarYear'])

# Chaque indicateur est comparé à sa version année n+1 et un ratio est fait pour en retirer la croissance annuelle
for indicator in indicators: # indicators définit au moment de l'affichage de la distribution
    yoy_growth_col_name = f'{indicator}YoYGrowth'
    df_sorted[yoy_growth_col_name] = df_sorted.groupby('symbol')[indicator].pct_change() * 100

In [11]:
# On supprimme ceux qui ne sont pas pertinents sous la forme croissance annuelle
df_sorted = df_sorted.drop(['marketCapYoYGrowth', 'peRatioYoYGrowth', 'shareValueYoYGrowth'], axis=1)

In [12]:
# Enregistrement du csv
df_sorted.to_csv('../csv/clean_nasdaq_data_opa.csv', index=False)

# Comparaison du P/E Ratio et de la croissance annuelle des bénéfices 
Une croissance plus élevée qu'un P/E ratio peut indiquer une opportunité boursière.  
On remarque une certaine linéarité dans les graphs

In [13]:
# Ticker à analyser - remplacer pour choisir l'entreprise
company_symbol = 'AAPL'
df_company = df_sorted[df_sorted['symbol'] == company_symbol].copy()

# Création d'une figure avec un second axe y 
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Ajout d'un graph pour le P/E Ratio
fig.add_trace(
    go.Scatter(x=df_company['calendarYear'], y=df_company['peRatio'], name="P/E Ratio", mode='lines+markers'),
    secondary_y=False,
)

# Ajout d'un graph pour le EPS Growth
fig.add_trace(
    go.Scatter(x=df_company['calendarYear'], y=df_company['netIncomePerShareYoYGrowth'], name="Croissance annuelle EPS (%)", mode='lines+markers'),
    secondary_y=True,
)

# Mise en page
fig.update_layout(
    title_text=f"{company_symbol} - Tendance du P/E Ratio vs. Croissance de l'EPS"
)
fig.update_xaxes(title_text="Year")
fig.update_yaxes(title_text="<b>P/E Ratio</b>", secondary_y=False)
fig.update_yaxes(title_text="<b>Croissance annuelle EPS (%)</b>", secondary_y=True)

fig.show(renderer='iframe') # à changer en .show('notebook') ou .show() si non fonctionnel

# Visualisation de l'ensemble des timeseries du dataset
Visualisation interactive

In [14]:
# Définition des métriques pour l'affichage interactif
metrics_to_choose = ['marketCap', 
                     'marginProfit', 
                     'peRatio', 
                     'roe', 
                     'roic',
                     'debtToEquity',
                     'shareValue',
                     'marginProfitYoYGrowth',
                     'roeYoYGrowth',
                     'roicYoYGrowth',
                     'revenuePerShareYoYGrowth',
                     'netIncomePerShareYoYGrowth',
                     'debtToEquityYoYGrowth']							


# Dictionnaire pour les labels et titres associés
metric_labels = {
    'marketCap'                  : 'Capitalisation boursière',
    'marginProfit'               : 'Marge nette',
    'peRatio'                    : 'P/E Ratio',
    'roe'                        : 'Rentabilité des fonds propres (%)',
    'roic'                       : 'Rentabilité des capitaux investis (%)',
    'debtToEquity'               : 'D/E Ratio',
    'shareValue'                 : "Valeur de l'action",
    'marginProfitYoYGrowth'      : 'Croissance annuelle de la marge nette (%)',
    'roeYoYGrowth'               : 'Croissance annuelle du ROE (%)',
    'roicYoYGrowth'              : 'Croissance annuelle du ROIC (%)',
    'revenuePerShareYoYGrowth'   : "Croissance annuelle du chiffre d'affaires par action (%)",
    'netIncomePerShareYoYGrowth' : 'Croissance annuelle des bénéfices nets (%)',
    'debtToEquityYoYGrowth'      : 'Croissance annuelle du D/E Ratio (%)',
}

In [15]:
# --- Préparation des données ---
companies = ['AAPL', 'META', 'NVDA', 'AMZN', 'NASDAQ'] # Ajouter tous les tickers désirés
filtering = df_sorted['symbol'].isin(companies)
df_to_plot = df_sorted[filtering].copy()

# NVDA est décalé d'un an (bilans en janvier)
condition = df_to_plot['symbol'] == 'NVDA'
df_to_plot.loc[condition, 'calendarYear'] -= 1 # Réaligne Nvidia

# --- Création du Graphique Initial ---
initial_metric = metrics_to_choose[0]
initial_label = metric_labels[initial_metric]

fig = px.line(
    df_to_plot, 
    x='calendarYear',
    y=initial_metric,
    color='symbol', 
    title=f"Comparaison: {initial_label} de 2020 à 2024",
    labels={initial_metric: initial_label},
    markers=True,
    height=600
)

# --- Création des Boutons pour le Menu de Métriques ---
buttons_metric = []
for metric in metrics_to_choose:
    label = metric_labels[metric]
    y_data_update = [df_to_plot[df_to_plot['symbol'] == company][metric].tolist() for company in companies]
    hover_template = f"<b>{label}</b>: %{{y}}<br>Year: %{{x}}<br>Symbol: %{{fullData.name}}<extra></extra>"
    buttons_metric.append(
        dict(
            method='update',
            label=label,
            args=[
                {'y': y_data_update, 
                 'hovertemplate': [hover_template] * len(companies)},
                {'title.text': f"Comparaison: {label} de 2020 à 2024",
                 'yaxis.title.text': label}
            ]
        )
    )

# --- Ajout du Menu et Mise en Page ---
fig.update_layout(
    updatemenus=[
        dict(
            # Menu pour les métriques
            buttons=buttons_metric,
            direction="down",
            pad={"r": 10, "t": 10},
            showactive=True,
            x=0.8, xanchor="left",
            y=1.15, yanchor="top"
        )
    ],
    xaxis_title="Year",
    yaxis_title=initial_label,
    legend_title_text='Tickers (Cliquer pour afficher/masquer)',
    showlegend=True
)

fig.update_xaxes(dtick=1)
fig.show(renderer='iframe')

# Suite du projet 
Il conviendre ensuite de se pencher sur une analyse métier des graphs ci dessus ainsi qu'un comparatif de corrélation des time series